### 2021_11_11_4

In [1]:
import os

In [2]:
!ls -al "../DATA/dogs-vs-cats/train/" | head -5
!ls -l "../DATA/dogs-vs-cats/train/" | grep ^- | wc -1
!ls -al "../DATA/dogs-vs-cats/test" | head -5
!ls -l "../DATA/dogs-vs-cats/test" | grep ^- | wc -1

total 1217008
drwxr-xr-x@ 25002 seungwoomun  staff  800064  9 20  2013 .
drwx------@     9 seungwoomun  staff     288 11 11 10:27 ..
-rw-r--r--@     1 seungwoomun  staff   12414  9 20  2013 cat.0.jpg
-rw-r--r--@     1 seungwoomun  staff   16880  9 20  2013 cat.1.jpg
wc: illegal option -- 1
usage: wc [-clmw] [file ...]
ls: ../DATA/dogs-vs-cats/test: No such file or directory
ls: ../DATA/dogs-vs-cats/test: No such file or directory
wc: illegal option -- 1
usage: wc [-clmw] [file ...]


In [8]:
# 원본 데이터셋을 압축 해제한 디렉터리 경로
ori_dataset_dir = '../DATA/dogs-vs-cats/train'

# 소규모 데이터셋을 저장할 디렉터리
base_dir = '../DATA/dogs-vs-cats/dogs_and_cats_small'

In [9]:
# 훈련, 검증, 테스트 분할을 위한 디렉터리
train_dir = os.path.join(base_dir, 'train')

val_dir = os.path.join(base_dir, 'validation')

test_dir = os.path.join(base_dir, 'test')

# 훈련용 고양이 사진 디렉터리
train_cats_dir = os.path.join(train_dir, 'cats')

# 훈련용 강아지 사진 디렉터리
train_dogs_dir = os.path.join(train_dir, 'dogs')

# 검증용 고양이 사진 디렉터리
val_cats_dir = os.path.join(val_dir, 'cats')

# 검증용 강아지 사진 디렉터리
val_dogs_dir = os.path.join(val_dir, 'dogs')

# 테스트용 고양이 사진 디렉터리
test_cats_dir = os.path.join(test_dir, 'cats')

# 테스트용 강아지 사진 디렉터리
test_dogs_dir = os.path.join(test_dir, 'dogs')

### [훈련/검증/테스트]에 들어있는 사진의 개수를 카운트

In [10]:
print('훈련용 고양이 이미지 전체 개수:', len(os.listdir(train_cats_dir)))
print('훈련용 강아지 이미지 전체 개수:', len(os.listdir(train_dogs_dir)))

print('검증용 고양이 이미지 전체 개수:', len(os.listdir(val_cats_dir)))
print('검증용 강아지 이미지 전체 개수:', len(os.listdir(val_dogs_dir)))

print('테스트용 고양이 이미지 전체 개수:', len(os.listdir(test_cats_dir)))
print('테스트용 강아지 이미지 전체 개수:', len(os.listdir(test_dogs_dir)))

훈련용 고양이 이미지 전체 개수: 1000
훈련용 강아지 이미지 전체 개수: 1000
검증용 고양이 이미지 전체 개수: 500
검증용 강아지 이미지 전체 개수: 500
테스트용 고양이 이미지 전체 개수: 500
테스트용 강아지 이미지 전체 개수: 500


### 사전 훈련 네트워크 가져오기

In [11]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights = "imagenet", 
                  include_top = False, # 완전 연결 계층(FCL)
                  input_shape = (150, 150, 3))

Using TensorFlow backend.
2021-11-11 13:26:08.273520: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


58892288/58889256 [==============================] - 5s 0us/step


* VGG16 함수에 세 개의 매개변수를 전달
    * weights는 모델을 초기화할 가중치 체크포인트를 지정
        * None, 'imagenet', weights의 path
    * include_top은 네트워크의 최상위 완전 연결 분류기를 포함할지 안할지를 지정.
        * 기본값은 ImageNet의 1,000개의 클래스에 대응되는 완전 연결 분류기를 포함.
        * 여기서는 별도의 (강아지와 고양이 두 개의 클래스를 구분하는) 완전 연결 층을 추가할 계획.
    * input_shape은 네트워크에 주입할 이미지 텐서의 크기입니다.
        * 이 매개변수는 선택사항입니다. 이 값을 지정하지 않으면 네트워크가 어떤 크기의 입력도 처리할 수 있습니다.

In [12]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

* VGG16은 합성곱 13개, 완전 연결 층(FCL) 3개.
    * include_top이 기본값이 True일 경우 input_shape이 원본 모델과 동일한 (224, 224, 3)이 되어야 한다.
* 다른 모델 VGG19는 합성곱 층 16개, 완전 연결 층 3개로 이루어져 있음.

### 데이터 증식을 사용하지 않는 빠른 특성 추출

In [19]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

# 소규모 데이터셋을 저장할 디렉터리
base_dir = '../DATA/dogs-vs-cats/dogs_and_cats_small'

# 데이터 디렉터리
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

### 이미지 제너레이터를 활용한 데이터 가져오기

In [20]:
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))

    print("피처, 레이블 : ", features.shape, labels.shape  )

    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        #print(  type(features_batch), features_batch.shape  )
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # 제너레이터는 루프 안에서 무한하게 데이터를 만들어내므로 
            # 모든 이미지를 한 번씩 처리하고 나면 중지합니다
            break
    #print(  type(features), features_batch.shape  )
    return features, labels

In [21]:
t_features, t_labels = extract_features(train_dir, 100)
print( t_features.shape, t_labels.shape )

피처, 레이블 :  (100, 4, 4, 512) (100,)
Found 2000 images belonging to 2 classes.
(100, 4, 4, 512) (100,)
